# Example Usage

In [1]:
import pathlib

import pandas as pd

## Creating Estimates Tables (generate_tables.EstimatesTables)

In [2]:
import generate_tables as gt

In [3]:
# Creating consolidated files using the vintage 2020_06 at the region and jurisdiction level
# "_ =" to supress output
_ = gt.EstimatesTables().consolidate("2020_06", 
        geo_list=["region", "jurisdiction"], 
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

In [4]:
# Creating individual files using the vintage 2020_06 at the region and jurisdiction level for the
# Estimates tables household_income and age_ethnicity
_ = gt.EstimatesTables().individual("2020_06",
        geo_list=["region", "jurisdiction"],
        est_table_list=["household_income", "age_ethnicity"],
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

## Creating Diff Files (generate_tables.DiffFiles)

In [5]:
# First create consolidated and individual files using the same parameters as before but with 
# a vintage of 2021_01
_ = gt.EstimatesTables().consolidate("2021_01", 
        geo_list=["region", "jurisdiction"], 
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))
_ = gt.EstimatesTables().individual("2021_01",
        geo_list=["region", "jurisdiction"],
        est_table_list=["household_income", "age_ethnicity"],
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

In [6]:
# Create diff files 
_ = gt.DiffFiles().create_diff_tables("2020_06", "2021_01", 
        geo_list=["region", "jurisdiction"],
        est_table_list=["consolidated", "household_income", "age_ethnicity"],
        raw_data_folder=pathlib.Path("./data/raw_data/"),
        save_folder=pathlib.Path("./data/diff/"))